## Описание проекта

Из открытых источников доступны исторические данные о продажах игр интернет-магазина «Стримчик», который продаёт по всему миру компьютерные игры,
оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation).
Нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными.
Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.
В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр.
ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».


## Цель проекта

Выявить определяющие успешность игры закономерности, с целью сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

### Описание данных

•	Name — название игры
•	Platform — платформа
•	Year_of_Release — год выпуска
•	Genre — жанр игры
•	NA_sales — продажи в Северной Америке (миллионы проданных копий)
•	EU_sales — продажи в Европе (миллионы проданных копий)
•	JP_sales — продажи в Японии (миллионы проданных копий)
•	Other_sales — продажи в других странах (миллионы проданных копий)
•	Critic_Score — оценка критиков (максимум 100)
•	User_Score — оценка пользователей (максимум 10)
•	Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.


## Шаг 1. Знакомство с данными

In [2]:
# импортируем необходимые библиотеки

import pandas as pd
import numpy as np
import re
import plotly
import plotly.graph_objs as go
import plotly.express as px
from scipy import stats as st

%matplotlib inline

In [3]:
# загрузим данные
data = pd.read_csv('../data/games.csv')
# дадим датафрейму имя для дальнейшего удобства использования
data.name = 'games'
# преобразуем названия колонок в нижний регистр
data.columns = [x.lower() for x in data.columns.to_list()]
# выведем датафрейм и информацию по нему на экран
display(
    data.head(),
    data.describe(),
    data.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


,year_of_release,na_sales,eu_sales,jp_sales,other_sales,critic_score
count,16446.000000,16715.000000,16715.000000,16715.000000,16715.000000,8137.000000
mean,2006.484616,0.263377,0.145060,0.077617,0.047342,68.967679
std,5.877050,0.813604,0.503339,0.308853,0.186731,13.938165
min,1980.000000,0.000000,0.000000,0.000000,0.000000,13.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,60.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,71.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,79.000000
max,2016.000000,41.360000,28.960000,10.220000,10.570000,98.000000


None

Датафрейм состоит из 16715 строк и 11 колонок, 6 из которых типа float и 5 типа Object.
В данных содержится информация по играм, выпущенным с 1980 по 2016 год, с оценками критиков от 13 до 98.
Посмотрим на пропуски в данных

In [4]:
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    data_len = len(data)
    # объявляем счетчик
    counter = -1
    display('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        display(f'Количество записей в датафрейме {data.name}: {data_len}')
        display(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                display(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
    else:
        display(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(data)

'============================================================'

'Количество записей в датафрейме games: 16715'

'В датафрейме games имеются следующие пропуски:'

'  - в колонке name: 2 пропусков, это 0.01% об общего объема данных'

'  - в колонке year_of_release: 269 пропусков, это 1.61% об общего объема данных'

'  - в колонке genre: 2 пропусков, это 0.01% об общего объема данных'

'  - в колонке critic_score: 8578 пропусков, это 51.32% об общего объема данных'

'  - в колонке user_score: 6701 пропусков, это 40.09% об общего объема данных'

'  - в колонке rating: 6766 пропусков, это 40.48% об общего объема данных'

## Шаг 2. Предобработка данных

У нас есть 2 пропуска в колонке name и genre. Скорее всего они в одних и тех же строках, убедимся в этом.

In [5]:
data[data['name'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Да, оба пропуска находятся в одинаковых строках. Удалим их.

In [6]:
data.drop(index=[659, 14244], inplace=True)
# убедимся, что строки удалены
data[data['name'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


Преобразуем типы данных

In [7]:
def auto_change_dtypes(data: pd.DataFrame) -> None:
    """
    Автоматически определяет тип столбца, и изменяет его в соответствии с хранимыми значениями.
    Функция не возвращает новый датафрейм, а изменяет переданный в качестве аргумента.
    Функция заточена под данные конкретного проекта.
    Функция поддерживает автоматическое преобразование следующих типов и форматов данных:
     - int64
     - float64
     - str: если в названии колонки есть date и формат даты %Y-%m-%d,
            то переводит в формат pandas datetime, иначе - переводит строковые данные в нижний регистр

    Пример преобразования:
    data[column] является int64 и содержит значения в диапазоне от 0 до 100 - будет преобразован в int8
    data[date_column] является object и содержит в имени колонки date - будет преобразован в datetime

    :param data: pd.DataFrame
    :return: None
    """
    # получаем количество используемой датафреймом памяти
    memory_usage_before_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    # получаем описание датафрейма
    describe = data.describe()
    # получаем названия колонок
    columns = data.columns.to_list()
    # получаем типы данных
    dtypes = data.dtypes
    # количество типов данных
    indexes = len(dtypes)
    # создаем 2 словаря для int и float, содержащие в качестве ключей типы данных, а значений - список из min и max значений этих типов данных
    correct_int_dtypes = {'int8': [-2**7, 2**7-1], 'int16': [-2**15, 2**15-1], 'int32': [-2**31, 2**31-1]}
    correct_float_dtypes = {'float16': [-2.0**16, 2.0**16-1], 'float32': [-2.0**31, 2.0**31-1]}

    display(f'{"="*30} Работаем с датафреймом {data.name} {"="*30}')
    # пробегаем по индексам колонок датафрейма и типам данных колонок
    for index, dtype in zip(range(0, indexes), dtypes):

        # если тип int64, меняем на тип, соответствующий значениям колонок
        if dtype == np.int64:
            for key, value in zip(correct_int_dtypes.keys(), correct_int_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип float64, меняем на тип, соответствующий значениям колонок
        elif dtype == np.float64:
            for key, value in zip(correct_float_dtypes.keys(), correct_float_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип object и колонка содержит в названии 'date' - меняем на datetime
        elif dtype == object:
            if 'date' in columns[index]:
                display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на datetime')
                data[columns[index]] = pd.to_datetime(data[columns[index]], format='%Y-%m-%d')
            # иначе приводим данные к нижнему регистру, пропуская значения nan (float) в колонке
            else:
                display(f'Приводим строковые данные в колонке {columns[index]} к нижнему регистру')
                data[columns[index]] = data[columns[index]].apply(lambda s: s if type(s) == float else s.lower())

    # количество памяти, используемое датафреймом оптимизации типов данных
    memory_usage_after_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    bytes_in_mb = 2**23
    display(f'Использование памяти датафрейма до сжатия: {(memory_usage_before_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Использование памяти датафрейма после сжатия: {(memory_usage_after_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Сжато: {((memory_usage_before_change_dtypes - memory_usage_after_change_dtypes) / bytes_in_mb):.2f} мб.')

auto_change_dtypes(data)

'============================== Работаем с датафреймом games =============================='

'Приводим строковые данные в колонке name к нижнему регистру'

'Приводим строковые данные в колонке platform к нижнему регистру'

'Изменяем тип колонки year_of_release датафрейма games с float64 на float16'

'Приводим строковые данные в колонке genre к нижнему регистру'

...

'Приводим строковые данные в колонке user_score к нижнему регистру'

'Приводим строковые данные в колонке rating к нижнему регистру'

'Использование памяти датафрейма до сжатия: 0.70 мб.'

'Использование памяти датафрейма после сжатия: 0.62 мб.'

'Сжато: 0.07 мб.'

Год релиза это точно целое число, так что преобразуем данную колонку из float16 в int16

In [8]:
data['year_of_release'] = pd.to_numeric(data['year_of_release']).convert_dtypes()
# проверим, что код отработал верно
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float16
 5   eu_sales         16713 non-null  float16
 6   jp_sales         16713 non-null  float16
 7   other_sales      16713 non-null  float16
 8   critic_score     8137 non-null   float16
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: Int64(1), float16(5), object(5)
memory usage: 1.1+ MB


Также видно, что в названиях некоторых игр есть год. Обычно год в названии игры совпадает с годом ее релиза,
а значит мы можем попробовать восстановить часть пропусков в колонке year_of_release спарсив из названия игры год релиза.
Проверим достоверность такого метода восстановления пропусков, посчитав количество названий с указанием года, где совпадает год релиза и где не совпадает.

In [9]:
# убедимся что в названии игры присутствует предполагаемый год релиза
data[data['year_of_release'].isna()].head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
183,madden nfl 2004,ps2,<NA>,sports,4.261719,0.260010,0.010002,0.709961,94.0,8.5,e
377,fifa soccer 2004,ps2,<NA>,sports,0.589844,2.359375,0.040009,0.509766,84.0,6.4,e
456,lego batman: the videogame,wii,<NA>,action,1.799805,0.970215,0.000000,0.290039,74.0,7.9,e10+
475,wwe smackdown vs. raw 2006,ps2,<NA>,fighting,1.570312,1.019531,0.000000,0.409912,NaN,NaN,NaN
609,space invaders,2600,<NA>,shooter,2.359375,0.140015,0.000000,0.029999,NaN,NaN,NaN


В играх также нередко бывает что год в названии игры следующий по отношению к году релиза, посчитаем заодно количество игр с такими названиями

In [10]:
def find_release_date(row: pd.Series, method='same') -> tuple:
    """
    Возвращает кортеж из названия игры и даты релиза по заданному условию.
    Варианты условий:
     - same - если дата в названии игры совпадает с годом релиза
     - one_year_different - если разница в дате в названии игры отличается на дату релиза на один год
     - deep_different - если разница в дате в названии игры отличается на дату релиза более чем на один год

    :param row: строка датафрейма с названием игры и годом релиза
    :type row: pd.Series
    :param method: условие для возвращаемых функцией значений
    :type method: str
    :return: название игры и год релиза или None
    :rtype: tuple
    """
    if not pd.isna(row['year_of_release']):
        if re.search('[1-2][0-9][0-9][0-9]', row['name']):
            match = int(re.search('[1-2][0-9][0-9][0-9]', row['name']).group())
            if method == 'same':
                if match == int(row['year_of_release']):
                    return row['name'], row['year_of_release']
            elif method == 'one_year_different':
                if match != int(row['year_of_release']) and (match - int(row['year_of_release'])) < 2:
                    return row['name'], row['year_of_release']
            elif method == 'deep_different':
                if match != int(row['year_of_release']) and (match - int(row['year_of_release'])) > 2:
                    return row['name'], row['year_of_release']

same = data[['name', 'year_of_release']].apply(find_release_date, method='same', axis=1).dropna().T.count()
one_year_different = data[['name', 'year_of_release']].apply(find_release_date, method='one_year_different', axis=1).dropna().T.count()
deep_different = data[['name', 'year_of_release']].apply(find_release_date, method='deep_different', axis=1).dropna().T.count()

display(f"Количество игр с совпадением года в названии с годом релиза: {same}")
display(f"Количество игр с отличием года в названии с годом релиза в единицу: {one_year_different}")
display(f"Количество игр со значительным (более 1 года) отличием года в названии с годом релиза: {deep_different}")
display(f"Вероятность угадать с названием при вставке даты релиза из названия: {same / (same + one_year_different + deep_different):.02%}")
display(f"Вероятность не угадать с разницей в один год с названием при вставке даты релиза из названия: {one_year_different / (same + one_year_different + deep_different):.02%}")
display(f"Вероятность совсем не угадать с названием при вставке даты релиза из названия (разница может быть более одного года): {deep_different / (same + one_year_different + deep_different):.02%}")


'Количество игр с совпадением года в названии с годом релиза: 363'

'Количество игр с отличием года в названии с годом релиза в единицу: 468'

'Количество игр со значительным (более 1 года) отличием года в названии с годом релиза: 29'

'Вероятность угадать с названием при вставке даты релиза из названия: 42.21%'

'Вероятность не угадать с разницей в один год с названием при вставке даты релиза из названия: 54.42%'

'Вероятность совсем не угадать с названием при вставке даты релиза из названия (разница может быть более одного года): 3.37%'

Построим на распределение количества совпадений и не совпадений года в названии по жанрам и по платформе

In [132]:
methods = ['same', 'one_year_different', 'deep_different']
differents_by_genre_data = pd.DataFrame()
differents_by_genre_platform = pd.DataFrame()
for method in methods:
    indexes = data[['name', 'year_of_release']].apply(find_release_date, method=method, axis=1).dropna().T.reset_index()['index']
    differents_by_genre_data[f'count_{method}'] = data.loc[indexes, :].groupby('genre').count().rename({'name': f'count_{method}'}, axis=1)[f'count_{method}']
    differents_by_genre_platform[f'count_{method}'] = data.loc[indexes, :].groupby('platform').count().rename({'name': f'count_{method}'}, axis=1)[f'count_{method}']

fig = px.bar(
    differents_by_genre_data.fillna(0).astype(int).sort_values(by='count_same', ascending=False),
    text_auto=True,
    title="Распределение количества ошибок и совпадений в названиях с годом релиза по жанрам"
)

fig.update_layout(
    xaxis_title="Жанр",
    yaxis_title="Количество совпадений"
)

fig.show()

fig = px.bar(
    differents_by_genre_platform.fillna(0).astype(int).sort_values(by='count_same', ascending=False),
    text_auto=True,
    title="Распределение количества ошибок и совпадений в названиях с годом релиза по платформам"
)

fig.update_layout(
    xaxis_title="Платформа",
    yaxis_title="Количество совпадений"
)

fig.show()


Из распределения по жанрам и платформам видим, что вероятность ошибиться при присваивании года релиза из названия довольно велика, независимо от группировки данных.
Т.о. делаем вывод о том, что не будем пытаться заполнить пропуски и оставим данные как есть, тем более что восстановить получится всего 17 пропусков.

Посмотрим, есть ли явные дубликаты по столбцам name, platform и genre

In [12]:
data[data[['name', 'platform', 'genre']].duplicated()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
1591,need for speed: most wanted,x360,2005,racing,1.0,0.130005,0.020004,0.099976,83.0,8.5,t
4127,sonic the hedgehog,ps3,<NA>,platform,0.0,0.479980,0.000000,0.000000,43.0,4.1,e10+
11715,need for speed: most wanted,pc,2012,racing,0.0,0.059998,0.000000,0.020004,82.0,8.5,t
16230,madden nfl 13,ps3,2012,sports,0.0,0.010002,0.000000,0.000000,83.0,5.5,e


Найдено 4 явных дубликата. Удалим их.

In [13]:
# получим индексы явных дублей
indexes = data[data[['name', 'platform', 'genre']].duplicated()].index.to_list()
# удалим дубли по индексу и пересчитаем индекс
data = data.drop(index=indexes).reset_index(drop=True)

Согласно ESRB рейтинг K-A (Kids to Adults) с 1998 года переименован в E (Everyone), соответственно переименуем рейтинг старых игр в E, чтобы не было путаницы.

In [15]:
# получим индексы записей с рейтингом K-A
indexes = data[data['rating'] == 'k-a'].index.to_list()
# присвоим рейтингу этих игр значение E
data.loc[indexes, 'rating'] = 'E'
# проверим, что все ок
data[data['rating'] == 'k-a']['name'].count()

0

Посчитаем суммарные продажи во всех регионах и запишем их в отдельный столбец.

In [16]:
data['sum_sales'] = data['na_sales'] + data['eu_sales'] + data['jp_sales'] + data['other_sales']

Рейтинг tbd (to be defined) в поле user_score скорее всего обозначает, что было слишком мало продаж для составления пользовательского рейтинга.
Посмотрим на продажи игр с таким рейтингом.

In [17]:
pd.DataFrame(
    data = [
    data[data['user_score'] == 'tbd']['sum_sales'].describe(),
    data[data['user_score'] != 'tbd']['sum_sales'].describe()
    ]
).reset_index(drop=True).rename({0: 'tbd_rating_sales', 1: 'not_tbd_rating_sales'})

,count,mean,std,min,25%,50%,75%,max
tbd_rating_sales,2424.0,0.188110,0.293701,0.010002,0.049988,0.109985,0.219971,6.699219
not_tbd_rating_sales,14285.0,0.591797,1.663086,0.000000,0.059998,0.189941,0.530273,82.500000


Действительно, суммарные значения в колонках продаж не такие большие, как в данных с рейтингом. Оставим рейтинг как есть, и будем иметь это в виду.

## Шаг 3. Исследовательский анализ данных.

Посмотрим, сколько игр выпускалось в разные годы.

In [18]:
fig = px.bar(
    data.groupby('year_of_release').count().rename({'name': 'count'}, axis=1)['count'],
    title='Количество выпущенных игр в разные годы',
    text_auto=True
)

fig.update_layout(
    xaxis_title="Год",
    yaxis_title="Количество игр"
)

fig.show()

Количество выпущенных игр начало плавно расти после 1992 года. Хоть игр до 1992 года выпускалось не много, среди них все равно есть очень успешные, которые популярны по сей день :)
Так что оставим эти данные для дальнейшей работы.

Посмотрим, как менялись продажи по платформам.

In [19]:
fig = px.bar(
    data.groupby('platform').sum()['sum_sales'].sort_values(ascending=False),
    title='Количество проданных игр на разных платформах',
    text_auto=True
)

fig.update_layout(
    xaxis_title="Платформа",
    yaxis_title="Количество проданных игр (млн. копий)"
)

fig.show()

Выберем платформы с наибольшими суммарными продажами и построим распределение по годам.

In [20]:
sum_sales_data = (data
                  .query("platform in ['ps2', 'x360', 'ps3', 'wii', 'ds', 'ps']")
                  .groupby(['platform', 'year_of_release'])
                  .sum()
                  .sort_values(by='sum_sales', ascending=False)
                  .reset_index())

fig = px.bar(
    x=sum_sales_data['year_of_release'],
    y=sum_sales_data['sum_sales'],
    color=sum_sales_data['platform'],
    title='Количество проданных игр на разных платформах'
)

fig.update_layout(
    xaxis_title="Год",
    yaxis_title="Количество проданных игр (млн. копий)"
)

fig.show()

Из распределения видно, что в среднем жизненный цикл платформы состовляет около 10 лет.
Возьмем данные за актуальный период, для выялвения перспективных платформ на 2017 год.
Учитывая жизненный цикл платформы в 10 лет, определим актуальный период с 2007 по 2016 годы, и посмотрим какие платформы лидируют по продажам, какие растут, а какие падают?
Выберем несколько потенциально прибыльных платформ.

In [21]:
actual_data = data.query("year_of_release >= 2007")

sales = actual_data.groupby(['platform', 'year_of_release']).sum().reset_index()

fig = px.bar(
    x=sales['year_of_release'],
    y=sales['sum_sales'],
    color=sales['platform'],
    title='Количество проданных игр на разных платформах'
)

fig.update_layout(
    xaxis_title="Год",
    yaxis_title="Количество проданных игр (млн. копий)"
)

fig.show()

Из графика видно что продажи таких платформ как 3ds, ds, pc, ps2, ps3, psp, psv, wii, wiiu, x360 падают.
Набирают популярность: ps4, xone.

Построим boxplot по глобальным продажам игр в разбивке по платформам.

In [134]:
fig = px.box(
    actual_data[['platform', 'sum_sales']],
    x='sum_sales',
    y='platform',
    orientation='h', # такой вид графика позволит гибко масштабировать отображаемые данные
    height=720,
    title='Глобальные продажи игр на разных платформах'
)

fig.update_layout(
    xaxis_title="Количество проданных игр (млн. копий)",
    yaxis_title="Платформа"
)

fig.show()

Из данного распределения видно, что за выбранный актуальный период больше всего продавались игры дл gba, и всех поколений XBOX и PS.

## Шаг 4. Портрет пользователя каждого региона

Выделим самые популярные платформы (топ-5)


In [23]:
regions = {
    'na_sales': 'Северной Америке',
    'eu_sales': 'Европе',
    'jp_sales': 'Японии',
    'other_sales': 'других странах'
}

t = actual_data.groupby('platform').sum().reset_index().sort_values(by='sum_sales', ascending=False)

for region, hr_region in regions.items():
    fig = px.bar(
        x=t['platform'],
        y=t[region],
        title=f'Количество проданных игр на разных платформах в {hr_region}'
    )

    fig.update_layout(
        xaxis_title="Платформа",
        yaxis_title="Количество проданных игр (млн. копий)"
    )

    fig.show()

Топ-5 игр в Северной Америке:
 - ps3 (381 млн. копий)
 - x360 (553 млн. копий)
 - wii (416 млн. копий)
 - ds (274 млн. копий)
 - ps4 (108 млн. копий)

Топ-5 игр в Европе:
 - ps3 (319 млн. копий)
 - x360 (360 млн. копий)
 - wii (214 млн. копий)
 - ds (119 млн. копий)
 - ps4 (141 млн. копий)

Топ-5 игр в Японии:
 - ds (103 млн. копий)
 - 3ds (100 млн. копий)
 - ps3 (78 млн. копий)
 - psp (66 млн. копий)
 - wii (60 млн. копий)

 Топ-5 игр в других странах:
 - ps3 (131 млн. копий)
 - x360 (80 млн. копий)
 - wii (64 млн. копий)
 - ps2 (54 млн. копий)
 - ps4 (48 млн. копий)

Судя по рейтингу топ-5 самых продаваемых игр потребители в Японии значительно отличаются по предпочтениям от всех,
там люди предпочитают мобильность и покупают не стационарные игровые платформы.
В Северной Америке и Европе больше всего покупают x360
В других странах отдают предпочтение ps3.
PS4 вошла в топ5 всех стран кроме Японии.

Выделим самые популярные жанры (топ-5)

In [24]:
regions = {
    'na_sales': 'Северной Америке',
    'eu_sales': 'Европе',
    'jp_sales': 'Японии',
    'other_sales': 'других странах'
}

t = actual_data.groupby('genre').sum().reset_index().sort_values(by='sum_sales', ascending=False)

for region, hr_region in regions.items():
    fig = px.bar(
        x=t['genre'],
        y=t[region],
        title=f'Количество проданных игр по жанрам в {hr_region}'
    )

    fig.update_layout(
        xaxis_title="Жанр",
        yaxis_title="Количество проданных игр (млн. копий)"
    )

    fig.show()

Топ-5 жанров в Северной Америке:
 - action (492 млн копий)
 - shooter (350 млн копий)
 - sports (323 млн копий)
 - misc (259 млн копий)
 - role-playing (175 млн копий)

Топ-5 жанров в Европе:
 - action (335 млн копий)
 - shooter (230 млн копий)
 - sports (211 млн копий)
 - misc (127 млн копий)
 - role-playing (102 млн копий)

Топ-5 жанров в Японии:
 - role-playing (150 млн копий)
 - action (96 млн копий)
 - misc (48 млн копий)
 - sports (40 млн копий)
 - adventure (27 млн копий)

Топ-5 жанров в других странах:
 - action (126 млн копий)
 - sports (82 млн копий)
 - shooter (81 млн копий)
 - misc (52 млн копий)
 - role-playing (36 млн копий)

В Северной и Европе америке лидируют по продажам шутеры, замыкают топ5 жанров ролевые игры.
В других странах лидируют по продажам экшн игры, замыкают топ5 жанров также ролевые игры.
В Японии же ролевые игры являются лидерами продаж, а замыкают топ5 жанров приключенческие игры.

Проверим, влияет ли рейтинг ESRB на продажи в отдельном регионе?
Для этого посчитаем и визуализируем коэффициенты корреляций для каждого региона рейтингом и продажами по региону.

In [86]:
for region, hr_region in regions.items():

    s_corr = actual_data['rating'].str.get_dummies().corrwith(actual_data[region]/t[region].max()).reset_index().rename(columns={'index': 'rating', 0: 'corr'})

    fig = px.bar(
        x=s_corr['rating'],
        y=s_corr['corr'],
        title=f'Корреляция между рейтингом и продажами в {hr_region}'
    )

    fig.update_layout(
        xaxis_title="Рейтинг",
        yaxis_title="Коеффициент корреляции Пирсона"
    )

    fig.show()

Как мы видим во всех регионах имеется небольшая зависимость между продажами игр с рейтингом M (18+), кроме Японии, там ни один рейтинг на продажи не влияет.

## Шаг 5. Проверим гипотезы

Есть предположение, что средние пользовательские рейтинги платформ Xbox One и PC одинаковые.

Из данного предположения формируем 2 гипотезы:
h0 - средние пользовательские рейтинги платформ Xbox One и PC одинаковые
h1 - средние пользовательские рейтинги платформ Xbox One и PC различаются

In [127]:
# делаем выборку пользовательских рейтингов по платформам, исключая tbd и пропуски
xone_ratings = actual_data.query("platform == 'xone' and user_score != 'tbd' and ~user_score.isna()")['user_score'].astype('float')
pc_ratings = actual_data.query("platform == 'pc' and user_score != 'tbd' and ~user_score.isna()")['user_score'].astype('float')
alpha = .05

res = st.ttest_ind(xone_ratings, pc_ratings)

if res.pvalue < alpha:
    print("Опровергаем нулевую гипотезу. Получены статистически значимые результаты, подтверждающие, что пользовательские рейтинги платформ Xbox One и PC различаются.")
else:
    print("Подтверждаем нулевую гипотезу. Получены статистически значимые результаты, подтверждающие, что пользовательские рейтинги платформ Xbox One и PC практически не различимы.")


Подтверждаем нулевую гипотезу. Получены статистически значимые результаты, подтверждающие, что пользовательские рейтинги платформ Xbox One и PC практически не различимы.


Также есть предположение, что средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

Из данного предположения формируем 2 гипотезы:
h0 - средние пользовательские рейтинги жанров Action и Sports одинаковые
h1 - средние пользовательские рейтинги жанров Action и Sports различаются

In [131]:
# делаем выборку пользовательских рейтингов по жанрам, исключая tbd и пропуски
action_ratings = actual_data.query("genre == 'action' and user_score != 'tbd' and ~user_score.isna()")['user_score'].astype('float')
sports_ratings = actual_data.query("genre == 'sports' and user_score != 'tbd' and ~user_score.isna()")['user_score'].astype('float')
alpha = .05

res2 = st.ttest_ind(action_ratings, sports_ratings)

if res2.pvalue < alpha:
    print("Опровергаем нулевую гипотезу. Получены статистически значимые результаты, подтверждающие, что пользовательские рейтинги жанров Action и Sports различаются.")
else:
    print("Подтверждаем нулевую гипотезу. Получены статистически значимые результаты, подтверждающие, что пользовательские рейтинги жанров Action и Sports практически не различимы.")

Опровергаем нулевую гипотезу. Получены статистически значимые результаты, подтверждающие, что пользовательские рейтинги жанров Action и Sports различаются.


## Общие выводы

В результате исследования можно сделать следующие выводы:
 - время жизни платформы ~10 лет
 - в Японии рейтинг не влияет на продажи игр, во всех остальных странах есть лишь не большая прямая зависимость от рейтинга M (18+)
 - наибольшие продажи в Японии приносят ролевые игры, во всех остальных странах наиболее популярными являются игры в жанраж Action, Shooter, Sports
 - пользователи в Японии предпочитают мобильные игровые консоли, в то время как пользователи других стран - стационарные
 - несмотря на одновременный старт, PS4 пользуется бОльшей популярностью, нежели XBOX One

Также проверили две гипотезы и получили:
 - статистически значимые результаты, подтверждающие, что пользовательские рейтинги платформ Xbox One и PC практически не различимы
 - статистически значимые результаты, подтверждающие, что пользовательские рейтинги жанров Action и Sports различаются